In [1]:
import numpy as np
from sklearn import datasets  # For a sample dataset
from collections import Counter

class KNN:
    def __init__(self, k=3, distance_metric='euclidean', weighted=False):
        self.k = k
        self.distance_metric = distance_metric
        self.weighted = weighted

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def _calculate_distance(self, x1, x2):
        if self.distance_metric == 'euclidean':
            return np.linalg.norm(x1 - x2)
        elif self.distance_metric == 'manhattan':
            return np.sum(np.abs(x1 - x2))
        else:
            raise ValueError("Invalid distance metric")

    def _get_neighbors(self, x):
        distances = [self._calculate_distance(x, x_train) for x_train in self.X_train]
        sorted_indices = np.argsort(distances)
        return sorted_indices[:self.k]

    def _predict_single(self, x):
        neighbors_idx = self._get_neighbors(x)
        neighbors_labels = self.y_train[neighbors_idx]

        if self.weighted:
            distances = [self._calculate_distance(x, self.X_train[idx]) for idx in neighbors_idx]
            weights = 1.0 / np.array(distances)  # Inverse weighting
            vote_counts = Counter(neighbors_labels * weights)  
        else:
            vote_counts = Counter(neighbors_labels)  

        return vote_counts.most_common(1)[0][0]

    def predict(self, X):
        return [self._predict_single(x) for x in X]

    def _accuracy(self, y_true, y_pred):
        return np.sum(y_true == y_pred) / len(y_true)
        
    # ... Other potential custom metrics like precision, recall, etc.

# **Sample Usage**
if __name__ == '__main__':
    iris = datasets.load_iris()
    X = iris.data
    y = iris.target


    knn = KNN(k=5, distance_metric='euclidean', weighted=True)
    knn.fit(X, y)

    y_pred = knn.predict(X) 
    accuracy = knn._accuracy(y, y_pred)
    print("Accuracy:", accuracy)


Accuracy: 0.3333333333333333


<ipython-input-1-55e82809798a>:34: RuntimeWarning: divide by zero encountered in divide
  weights = 1.0 / np.array(distances)  # Inverse weighting
<ipython-input-1-55e82809798a>:35: RuntimeWarning: invalid value encountered in multiply
  vote_counts = Counter(neighbors_labels * weights)


In [2]:
class KDNode:
    def __init__(self, point, label, left=None, right=None, axis=0):
        self.point = point
        self.label = label
        self.left = left
        self.right = right
        self.axis = axis

class KDTree:
    def __init__(self, points, labels):
        self.root = self.build_tree(points, labels)
    
    def build_tree(self, points, labels, depth=0):
        if len(points) == 0:
            return None
        
        # Select axis based on depth so that axis cycles over all valid values
        axis = depth % len(points[0])
        
        # Sort point list and choose median as pivot element
        sorted_points = sorted(zip(points, labels), key=lambda x: x[0][axis])
        median = len(sorted_points) // 2
        
        # Create node and construct subtrees
        return KDNode(
            point=sorted_points[median][0],
            label=sorted_points[median][1],
            left=self.build_tree([x[0] for x in sorted_points[:median]], [x[1] for x in sorted_points[:median]], depth + 1),
            right=self.build_tree([x[0] for x in sorted_points[median+1:]], [x[1] for x in sorted_points[median+1:]], depth + 1),
            axis=axis
        )

    def nearest(self, point, k=1):
        best = []
        self._search_tree(self.root, point, k, best)
        return sorted(best, key=lambda x: x[0])[:k]
    
    def _search_tree(self, node, point, k, best):
        if node is None:
            return
        
        # Compute distance from point to current node
        dist = euclidean_distance(point, node.point)
        
        if len(best) < k or dist < best[0][0]:
            best.append((dist, node.label))
            best.sort(reverse=True)
            if len(best) > k:
                best.pop(0)
        
        # Check which subtree to search
        if point[node.axis] < node.point[node.axis]:
            self._search_tree(node.left, point, k, best)
            if len(best) < k or abs(point[node.axis] - node.point[node.axis]) < best[0][0]:
                self._search_tree(node.right, point, k, best)
        else:
            self._search_tree(node.right, point, k, best)
            if len(best) < k or abs(point[node.axis] - node.point[node.axis]) < best[0][0]:
                self._search_tree(node.left, point, k, best)


In [3]:
import numpy as np

def euclidean_distance(a, b):
    return np.sqrt(np.sum((np.array(a) - np.array(b)) ** 2))


In [4]:
def knn_classify(kdtree, point, k=3):
    neighbors = kdtree.nearest(point, k)
    # Perform majority vote
    votes = {}
    for _, label in neighbors:
        votes[label] = votes.get(label, 0) + 1
    return max(votes, key=votes.get)


In [5]:
def accuracy(y_true, y_pred):
    correct = sum(1 for true, pred in zip(y_true, y_pred) if true == pred)
    return correct / len(y_true)

def confusion_matrix(y_true, y_pred, labels):
    matrix = np.zeros((len(labels), len(labels)), dtype=int)
    label_to_index = {label: i for i, label in enumerate(labels)}
    for true, pred in zip(y_true, y_pred):
        matrix[label_to_index[true]][label_to_index[pred]] += 1
    return matrix


In [6]:
def normalize(data):
    return (data - np.mean(data, axis=0)) / np.std(data, axis=0)


In [7]:
from sklearn import datasets  # Only for dataset loading

def load_and_split_data(test_ratio=0.2):
    iris = datasets.load_iris()
    data = normalize(iris.data)
    labels = iris.target
    indices = np.arange(len(data))
    np.random.shuffle(indices)
    split = int(len(data) * (1 - test_ratio))
    return data[indices[:split]], labels[indices[:split]], data[indices[split:]], labels[indices[split:]]

train_data, train_labels, test_data, test_labels = load_and_split_data()

# Build KD-Tree with training data
kdtree = KDTree(train_data, train_labels)

# Test the model
predictions = [knn_classify(kdtree, point, k=3) for point in test_data]

# Evaluate the model
print("Accuracy:", accuracy(test_labels, predictions))
print("Confusion Matrix:\n", confusion_matrix(test_labels, predictions, labels=np.unique(test_labels)))


Accuracy: 0.9666666666666667
Confusion Matrix:
 [[ 7  0  0]
 [ 0 12  0]
 [ 0  1 10]]
